Importação de Bibliotecas e Módulos

In [54]:
import pandas as pd #Manipulação de conjuntos de dados
import numpy as np #Manipulações numéricas
import matplotlib.pyplot as plt #Análises gráficas
import seaborn as sns #Análises gráficas
import io #Manipulação de funções do sistema
import os #Manipulação de funções do sistema
import sys #Manipulação de funções do sistema

sys.path.insert(0, os.getcwd()) #Inicialização da pasta do projeto

from recursos import teste, treino_trat, teste_trat, caminho #Importação dos Conjuntos
from sklearn.model_selection import train_test_split #Divisões dos conjuntos treino/teste
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score #Extração de indicadores
from xgboost import XGBRegressor  #Modelo de Aprendizado

Pré-processamento

In [55]:
#Importação dos Conjuntos e Refatoramento do Target
dfTreino = treino_trat()
dfTest = teste_trat()

dfTreino['preco'] = np.log10(dfTreino['preco'])

In [56]:
#Divisão dos conjuntos de Treino e Teste
x = dfTreino.drop(columns='preco')
y = dfTreino['preco']
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)

Implementação do Modelo

In [57]:
xgb = XGBRegressor().fit(X_train, y_train)
y_pred = xgb.predict(X_test)

Verificação dos Indicadores

In [58]:
data ={"r2" : r2_score(y_test, y_pred),
       "mse" : mean_squared_error(y_test, y_pred),
       "mae" : mean_absolute_error(y_test, y_pred)}

dfIndicadores = pd.Series(data)
dfIndicadores

r2     0.768437
mse    0.013997
mae    0.091147
dtype: float64

In [59]:
y_test = np.expm1(y_test)
y_pred = np.expm1(y_pred)
diffPercent =  (abs(y_test - y_pred) / y_test) * 100

dfResultados = pd.DataFrame({'Preço Real': y_test,
                             'Preço Predito': y_pred,
                             'Diferença Percentual': diffPercent})
dfResultados

,Preço Real,Preço Predito,Diferença Percentual
7936,189.016688,163.057007,13.734068
22472,121.170026,136.843292,12.934937
18848,154.042408,167.741821,8.893274
24252,275.059530,228.373978,16.972890
17123,158.605590,144.293106,9.023946
...,...,...,...
6357,139.019230,130.226517,6.324818
1135,138.655003,171.465881,23.663681
10359,107.799499,122.703819,13.825964
28967,316.852912,283.529633,10.516955


Aplicação do Modelo com o Conjunto de Teste

In [60]:
features = dfTest
price = xgb.predict(features)
price = np.expm1(price)

Exportação dos Resultados

In [61]:
PATH = caminho()

dfFinal = pd.DataFrame({'id': teste()['id'], 'preco': price})

resultdo = os.getcwd() + PATH + 'predict.csv'
dfFinal.to_csv(resultdo,sep="\t",encoding='UTF-16')